In [ ]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)


sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir)
    
import torch
from matplotlib import pyplot as plt

a = torch.rand(1,3,8,12)
N,C,H,W = a.size()
plt.imshow(a[0,0])
plt.colorbar()
plt.show()

In [ ]:
from torch import nn
conv_c = nn.Conv2d(C, 1, 1, bias=False)
bn = nn.BatchNorm2d(C)
# conv_h = nn.Conv2d(C, C, (1, W), bias=False, groups=C)
# conv_w = nn.Conv2d(C, C, (H, 1), bias=True, groups=C)
conv_h = nn.Conv2d(C, C, (1, W), bias=False)
conv_w = nn.Conv2d(C, C, (H, 1), bias=False)
selu = nn.SiLU()
sigmoid = nn.Sigmoid()
relu6 = nn.ReLU6()
# act = nn.Softmax(dim=2)

x_h = sigmoid(conv_h(a))
x_w = sigmoid(conv_w(a))

print(x_h.shape)
print(x_w.shape)
# print(torch.relu(x_h+x_w).shape)

result = relu6(conv_c(x_h*x_w)).detach()
# result = torch.softmax(result.view(N,1,-1), dim=2).view(N,1,W,H)
plt.imshow(result[0,0])
plt.colorbar()
plt.show()

In [ ]:
f = torch.mul(result.expand_as(a), a) 
plt.imshow(f[0,1])
plt.colorbar()
plt.show()

In [ ]:
f.shape

# 2nd Option

Average as spatial descriptor

In [ ]:
from torch import nn

n,c,w,h = a.size()

pool_h = nn.AdaptiveAvgPool2d((None, 1))
pool_w = nn.AdaptiveAvgPool2d((1, None))

conv_c = nn.Conv2d(c, 8, 1, bias=True)
bn = nn.BatchNorm2d(8)

# conv_h = nn.Conv2d(C, 1, (1, W), bias=True)
# conv_w = nn.Conv2d(C, 1, (H, 1), bias=True)
conv_h = nn.Conv2d(8, c, 1, bias=False)
conv_w = nn.Conv2d(8, c, 1, bias=False)

# Activation
silu = nn.SiLU()
act = nn.Softmax(dim=2)
sigmoid = nn.Sigmoid()
relu = nn.ReLU()

x_h_descriptor = pool_h(a)
x_w_descriptor = pool_w(a)

x_w_descriptor = x_w_descriptor.permute(0, 1, 3, 2)

test = torch.cat([x_w_descriptor, x_h_descriptor], dim=2)

# test = x_w_descriptor + x_h_descriptor 
# result = bn(silu(conv_c(test))).detach()
result = relu6(bn(conv_c(test))).detach()

x_w, x_h = torch.split(result, [h, w], dim=2)
x_w = x_w.permute(0, 1, 3, 2)

print(a.shape)
print(x_h.shape)
print(x_w.shape)

x_h = sigmoid(conv_h(x_h))
x_w = sigmoid(conv_w(x_w))
# x_h = (conv_h(x_h))
# x_w = (conv_w(x_w))

result = (a*x_w * x_h).detach()

# result = act(conv_c((x_h+x_w))).detach()
# # result = torch.softmax(result.view(N,1,-1), dim=2).view(N,1,W,H)
plt.figure(figsize=(16,4))
for i in range(c):
    plt.subplot(1,c,i+1)
    plt.imshow(result[0,i])
    plt.colorbar()

    
# plt.colorbar()
plt.show()

In [ ]:
print(result[result<.1].shape)
print(result[result>=.1].shape)

In [ ]:
from AttentionMap.LocalAttention import CoordinateAttentionBlock, CoordAttentionConv

test = CoordinateAttentionBlock(3, 16, 4)
test(a).shape

In [ ]:
result = test(a).detach()

print(result[result<.25].shape)
print(result[result>=.25].shape)

plt.figure(figsize=(16,4))
for i in range(result.size(1)):
    plt.subplot(2,result.size(1)//2,i+1)
    plt.imshow(result[0,i])
    plt.colorbar()

plt.show()

In [ ]:
model = CoordAttentionConv(3, 8, 8)

In [ ]:
result = model(a).detach()

plt.figure(figsize=(16,4))
for i in range(result.size(1)):
    plt.subplot(2,result.size(1)//2,i+1)
    plt.imshow(result[0,i])
    plt.colorbar()

plt.show()

# 1st Test

In [ ]:
from torch.nn import functional as F
class CoordAttentionBlock(nn.Module):
    def __init__(self, n_channels: int, img_size: tuple) -> None:
        super(CoordAttentionBlock, self).__init__()
        C = n_channels
        H, W = img_size
        reduction = 8

        out_channels = C // reduction
        
        self.conv_c = nn.Conv2d(out_channels, 1, 1, bias=False)
        # self.conv_h = nn.Conv2d(C, 1, (1, W), bias=False)
        # self.conv_w = nn.Conv2d(C, 1, (H, 1), bias=False)
        # self.conv_h = nn.Conv2d(C, C, (1, W), bias=False, groups=C)
        # self.conv_w = nn.Conv2d(C, C, (H, 1), bias=False, groups=C)
        self.conv_h = nn.Sequential(
            nn.Conv2d(C, out_channels, (1, W), bias=False, groups=out_channels),
            # nn.BatchNorm2d(C),
            # nn.SiLU()
            nn.ReLU(inplace=True)
        )
        self.conv_w = nn.Sequential(
            nn.Conv2d(C, out_channels, (H, 1), bias=False, groups=out_channels),
            # nn.BatchNorm2d(C),
            # nn.SiLU()
            nn.ReLU(inplace=True)
        )
        # self.act = nn.Softmax(1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        N,C,H,W = x.size()
        x_h = self.conv_h(x)
        x_w = self.conv_w(x)

        c = self.conv_c(x_h+x_w)
        # c = self.conv_c(x_h+torch.sigmoid(x_w))
        # c = torch.relu(x_h+x_w)
        
        a = torch.softmax(c.view(N,1,-1), dim=2).view(N,1,W,H)
        f = torch.mul(a.expand_as(x), x)
        output = f.view(N,C,-1).sum(dim=2)
        
        return c, output

In [ ]:
class ConvTestBlock(nn.Module):
    def __init__(self, in_channels, out_channels, img_size: tuple) -> None:
        super(ConvTestBlock, self).__init__()
        self.conv = nn.Sequential(*[
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ])

        self.coordAttBlock = CoordAttentionBlock(out_channels, img_size)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        attention = self.coordAttBlock(x)
        
        return torch.mul(attention.expand_as(x), x)

In [ ]:
class Model(nn.Module):
    def __init__(self, ) -> None:
        super(Model, self).__init__()
        self.l1 = nn.Sequential(*[
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])
        self.l2 = nn.Sequential(*[
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])
        self.l3 = nn.Sequential(*[
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])

        self.classifier = nn.Sequential(*[
            nn.Linear(64+128+256,32),
            nn.ReLU(),
            nn.Linear(32,10)
        ])

        img_size = 32
        self.coordAttBlock_l1 = CoordAttentionBlock(64, (int(img_size/2**1), int(img_size/2**1)) )
        self.coordAttBlock_l2 = CoordAttentionBlock(128, (int(img_size/2**2), int(img_size/2**2)) )
        self.coordAttBlock_l3 = CoordAttentionBlock(256, (int(img_size/2**3), int(img_size/2**3)) )

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x_l1 = self.l1(x)
        x_l2 = self.l2(x_l1)
        x_l3 = self.l3(x_l2)

        c1, g1 = self.coordAttBlock_l1(x_l1)
        c2, g2 = self.coordAttBlock_l2(x_l2)
        c3, g3 = self.coordAttBlock_l3(x_l3)

        g = torch.cat((g1,g2,g3), dim=1)
        y_hat = self.classifier(g)

        return (y_hat, c1, c2, c3)

In [ ]:
from AttentionMap.LocalAttention import CoordinateAttentionBlock
class Model(nn.Module):
    def __init__(self, ) -> None:
        super(Model, self).__init__()
        self.l1 = nn.Sequential(*[
            nn.Conv2d(3, 12, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])
        self.l2 = nn.Sequential(*[
            nn.Conv2d(12, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])
        self.l3 = nn.Sequential(*[
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ])

        self.classifier = nn.Sequential(*[
            nn.Linear(12+64+128,32),
            nn.ReLU(),
            nn.Linear(32,10)
        ])

        img_size = 32
        self.coordAttBlock_l1 = CoordAttentionBlock(12, (int(img_size/2**1), int(img_size/2**1)) )
        self.coordAttBlock_l2 = CoordAttentionBlock(64, (int(img_size/2**2), int(img_size/2**2)) )
        self.coordAttBlock_l3 = CoordAttentionBlock(128, (int(img_size/2**3), int(img_size/2**3)) )

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x_l1 = self.l1(x)
        x_l2 = self.l2(x_l1)
        x_l3 = self.l3(x_l2)

        c1, g1 = self.coordAttBlock_l1(x_l1)
        c2, g2 = self.coordAttBlock_l2(x_l2)
        c3, g3 = self.coordAttBlock_l3(x_l3)

        g = torch.cat((g1,g2,g3), dim=1)
        y_hat = self.classifier(g)

        return (y_hat, c1, c2, c3)

## Dataset

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST, CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize

transform = Compose([Resize(32), ToTensor()])
# train_dataset = MNIST('dataset/', train=True, transform=transform, download=True)
train_dataset = CIFAR10('dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# test_dataset = MNIST('dataset/', train=False, transform=transform, download=True)
test_dataset = CIFAR10('dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

img_size = 32

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as utils
from AttentionMap.utils import visualize_attention
from tqdm import tqdm

model = Model()
model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

tb_writer = SummaryWriter('log/CoordAttention/CIFAR')
running_avg_accuracy = 0
step = 0

# TMP
log_images = True
n_epoch = 50

epoch_iterator = tqdm(
        range(n_epoch),
        leave=True,
        unit="epoch",
        postfix={"tls": "%.4f" % 1},
    )

for epoch in epoch_iterator:
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        # log scalars
        images_disp = []
        for i, data in enumerate(test_loader, 0):
                images_test, labels_test = data
                images_test, labels_test = images_test.to(device), labels_test.to(device)
                pred_test, c1, c2, c3 = model(images_test)
                predict = torch.argmax(pred_test, 1)
                total += labels_test.size(0)
                correct += torch.eq(predict, labels_test).sum().double().item()
                
        tb_writer.add_scalar('test/accuracy', correct/total, epoch)

        n_rows=4
        n_images = 24
        activation = 'softmax'
        # C1
        scale_factor = 2**1
        vis = visualize_attention(n_rows, images_test[:n_images], c1[:n_images], scale_factor, activation=activation)
        tb_writer.add_image('Attention/C1', vis, epoch)

        # C2
        scale_factor = 2**2
        vis = visualize_attention(n_rows, images_test[:n_images], c2[:n_images], scale_factor, activation=activation)
        tb_writer.add_image('Attention/C2', vis, epoch)

        # C3
        scale_factor = 2**3
        vis = visualize_attention(n_rows, images_test[:n_images], c3[:n_images], scale_factor, activation=activation)
        tb_writer.add_image('Attention/C3', vis, epoch)

    model.train()
    for idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        inputs = inputs.to(device)
        targets = targets.to(device)
        pred, __, __, __ = model(inputs)

        loss = criterion(pred, targets)
        loss.backward()
        optimizer.step()

        if idx % 250 == 0:
            model.eval()
            pred, __, __, __ = model(inputs)
            predict = torch.argmax(pred, 1)
            total = targets.size(0)
            correct = torch.eq(predict, targets).sum().double().item()
            accuracy = correct / total
            running_avg_accuracy = 0.6*running_avg_accuracy + 0.4*accuracy
            tb_writer.add_scalar('train/loss', loss.item(), step)
            tb_writer.add_scalar('train/accuracy', accuracy, step)
            tb_writer.add_scalar('train/running_avg_accuracy', running_avg_accuracy, step)
            step += 1

            epoch_iterator.set_postfix(tls="%.4f" % loss.item())            

In [ ]:
plt.imshow(c1[0,0].detach().cpu())
plt.colorbar()
plt.show()
plt.imshow(c2[0,0].detach().cpu())
plt.colorbar()
plt.show()
plt.imshow(c3[0,0].detach().cpu())
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(input[0,0].detach().cpu())
plt.colorbar()
plt.show()

# Convolutional Attention Block

Se utiliza kernels de distintos tamaños para obtener descriptores horizontales y verticales, el propósito es obtener descriptores que definan la información espacial de forma complementaria para luego unificar dicha información mediante la adición (o concatenación) de ambos descriptores.

De todas formas, no se puede ignorar la información de los canales de cada input. Estos podrían afectar drásticamente a la los descriptores espaciales. Por este motivo, se han aplicado DepthWise Separable convolutions para que cada descriptor no se base toda su información teniendo en cuenta todos los canales, sino un subconjunto de ellos. (Es necesario desarrollar esto).

In [ ]:
class ConvolutionalAttentionBlock(nn.Module):
    def __init__(self, img_size: tuple, in_channels: int, reduction_rate: int, groups=False, bias=True) -> None:
        super(CoordinateAttentionBlock, self).__init__()
        out_channels = max(8, in_channels // reduction_rate)
        H, W = img_size

        self.conv_h = nn.Sequential(
            nn.Conv2d(C, out_channels, (1, W), bias=bias, groups=out_channels if groups else 1),
            nn.BatchNorm2d(out_channels),
            nn.SiLU()
        )

        self.conv_w = nn.Sequential(
            nn.Conv2d(C, out_channels, (H, 1), bias=bias, groups=out_channels if groups else 1),
            nn.BatchNorm2d(out_channels),
            nn.SiLU()
        )
        
        self.att = nn.Sequential(
            nn.Conv2d(out_channels, in_channels, 1, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_h = self.conv_h(x) # Height descriptor
        x_w = self.conv_w(x) # Weight descriptor

        # Coordinate attention
        coordAtt = self.att(x_h+x_w)
        # TODO: Concatenate x_h and x_w
        
        return coordAtt  

class CoordAttConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int, groups: int, bias: bool) -> None:
        super(CoordAttConv, self).__init__()

        self.conv = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )
        
        # self.att_block = CoordinateAttentionBlock(out_channels, out_channels, att_reduction)


In [205]:
import torchvision.models as models
resnet18 = models.resnet18()
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [1]:
from torchvision.models.resnet import conv3x3

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

NameError: name 'nn' is not defined

In [ ]:
models.ResNet()

In [ ]:
in_channels = 16
reduction_rate = 4
out_channels = in_channels // reduction_rate
a = torch.rand((1,in_channels,4,4))

In [ ]:
conv_h = nn.Conv2d(in_channels, out_channels, kernel_size=(1,4), groups=out_channels, bias=False)
conv_w = nn.Conv2d(in_channels, out_channels, kernel_size=(4,1), groups=out_channels, bias=False)

In [ ]:
silu = nn.SiLU()
desc_h = silu(conv_h(a))
desc_w = silu(conv_w(a))

desc = (desc_h + desc_w).detach()

conv_c = nn.Conv2d(out_channels, in_channels, kernel_size=(1,1), bias=False)
result = conv_c(desc).detach()

In [ ]:
# find minimum of minima & maximum of maxima
minmin = torch.min(result)

for i in range(out_channels):
    plt.subplot(out_channels, 1, i+1)
    plt.imshow(desc[0,i], vmin=torch.min(desc), vmax=torch.max(desc))
    plt.colorbar()

In [ ]:
plt.figure(figsize=(24,10))
# test = torch.softmax(result.flatten(2), dim=2)
# test = test.reshape(result.shape)
for i in range(in_channels):
    plt.subplot(1, in_channels, i+1)
    plt.imshow(torch.sigmoid(result[0,i]))
    plt.colorbar()

In [ ]:
test.sum(axis=1)

In [ ]:
test = torch.softmax(result.flatten(2), dim=2)
test = test.reshape(1,16,4,4)